#### AGENT
An agent take a self-determined, input-dependent sequence of steps before returning a user-facing output

First: Build an agent that has two tools: one to look things up online (Tavily), and one to look up specific data that we’ve loaded into a index using a retriever.

In [1]:
# Libraries

%pip install langchain
%pip install --upgrade --quiet  langchain-openai
%pip install langchainhub
%pip install BeautifulSoup4
%pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 19.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 5.3 MB/s eta 0:00:00

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__632ae1a420dc4b46b2894dd749f33b2e"

os.environ["TAVILY_API_KEY"] = "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"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://js-playground-01.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "43d936898d7645c3b3c1ebf294f04cd3"

#### Tavily Search engine tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'url': 'https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US',
  'content': "Today's and tonight's San Francisco, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com"},
 {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629',
  'content': 'San Francisco, CA Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'},
 {'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco',
  'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'},
 {'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA',
  'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire 

#### Retriever over data

In [4]:
# FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of documents that are similar to each other.

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment="ada-embedding", openai_api_version="2023-05-15")
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever.get_relevant_documents("how to upload a dataset")[0]

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [5]:
# Tool

tools = [search, retriever_tool]

In [6]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt4-32k",
)

In [8]:
from langchain import hub

# Get the prompt to use - you can modify this!

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [9]:
# Initialize an agent

from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [10]:
# AgentExecutor - repeatedly call the agent and execute tools

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [12]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith suppo

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith is a platform designed to help with building production-grade Language Models (LLM) applications. It provides several features that could be beneficial for testing:\n\n- **Debugging**: LangSmith allows you to debug your LLM applications, which can be essential during the testing phase to identify and fix issues.\n\n- **Evaluation**: LangSmith supports the evaluation of chains and intelligent agents built on any LLM framework. This can be used to assess the performance of your models during testing.\n\n- **Monitoring**: With LangSmith, you can monitor your applications, which can be useful for tracking their performance over time and identifying any potential issues that might arise during testing.\n\n- **Integration with LangChain**: LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. This can help streamline the testing process if you're using LangChain to build your ap